Live Code 3<br>
Nama: Risqi Wahyu Permana<br>
Batch: HCK 6

Menghubungkan dengan bigquery

In [6]:
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "hactiv8-risqi" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


##PROBLEM STATEMENT<br>
1. Apa saja kasus kejahatan kota Austin di tahun 2016?
2. Berapa banyak kasus kejahatan yang terjadi pada tahun 2016?
3. jenis kejahatan apa yang paling banyak terjadi pada tahun 2016?
4. Bulan apa yang kasusnya belum banyak terselesaikan?
5. Distrik apa saja yang paling banyak terjadi kasus pencurian
6. Berapa rata - rata waktu yang diperlukan untuk menyelesaiakn sebuah kasus?

## Penjabaran Masalah

### Problem 1
1. Tampilkan 5 data pertama pada tabel crime

In [7]:
df_status = client.query('''
  SELECT *
  FROM `bigquery-public-data.austin_crime.crime` LIMIT 5
''').to_dataframe()
df_status

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,2015150483,"Austin, TX",NaN,2015-01-27 12:00:00+00:00,Not cleared,<NA>,RAPE,B,NaN,NaN,None,nan,Rape,2015-01-15 12:00:00+00:00,<NA>,<NA>,2015,None
1,20151542027,"Austin, TX",NaN,2015-07-24 12:00:00+00:00,Not cleared,<NA>,RAPE,D,NaN,NaN,None,nan,Rape,2015-06-03 12:00:00+00:00,<NA>,<NA>,2015,None
2,20152001758,"Austin, TX",NaN,2015-08-15 12:00:00+00:00,Not cleared,<NA>,RAPE,F,NaN,NaN,None,nan,Rape,2015-07-19 12:00:00+00:00,<NA>,<NA>,2015,None
3,20151340282,"Austin, TX",NaN,2015-06-15 12:00:00+00:00,Cleared by Exception,<NA>,RAPE,UK,NaN,NaN,None,nan,Rape,2015-05-14 12:00:00+00:00,<NA>,<NA>,2015,None
4,20151871268,"Austin, TX",NaN,2015-10-27 12:00:00+00:00,Not cleared,<NA>,RAPE,UK,NaN,NaN,None,nan,Rape,2015-07-06 12:00:00+00:00,<NA>,<NA>,2015,None


### Problem 2
2. Apa saja jenis kejahatan pada tahun 2016 di kota Austin?

In [18]:
df_crime_2016 = client.query('''
  SELECT distinct primary_type, EXTRACT(YEAR FROM timestamp) as tahun
  FROM `bigquery-public-data.austin_crime.crime`
  WHERE EXTRACT(YEAR FROM timestamp) = 2016 #filter tahun 2016
  ''').to_dataframe()
df_crime_2016

,primary_type,tahun
0,Rape,2016
1,Theft,2016
2,Robbery,2016
3,Agg Assault,2016
4,Burglary,2016
5,Auto Theft,2016
6,Murder,2016


#### Insight

Berdasarkan data diatas terdapat `6` jenis kejahatan yang terjadi di kota Austin pada tahun 2016.

### Problem 3
3. Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?

In [20]:
df_cleared = client.query('''
  SELECT COUNT(description) as jumlah_kejahatan, EXTRACT(YEAR FROM clearance_date) as tahun
  FROM `bigquery-public-data.austin_crime.crime`
  WHERE EXTRACT(YEAR FROM clearance_date) = 2016
  GROUP BY tahun;
''').to_dataframe()
df_cleared

,jumlah_kejahatan,tahun
0,35747,2016


#### Insight

Terdapat `35.747` kejahatan yang memiliki kejelasan apakah kasus tersebut sudah terselesaikan atau belum

### Problem 4
4. Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?

In [21]:
df_not_cleared= client.query('''
  SELECT primary_type, COUNT(primary_type) as jumlah_kejahatan, EXTRACT(YEAR FROM clearance_date) as tahun
  FROM `bigquery-public-data.austin_crime.crime`
  WHERE EXTRACT(YEAR FROM clearance_date) = 2016 AND clearance_status = 'Not cleared'
  GROUP BY primary_type, tahun
  ORDER BY jumlah_kejahatan DESC;
''').to_dataframe()
df_not_cleared

,primary_type,jumlah_kejahatan,tahun
0,Theft,21945,2016
1,Burglary,4446,2016
2,Auto Theft,1494,2016
3,Agg Assault,899,2016
4,Robbery,559,2016
5,Rape,330,2016


#### Insight

Jenis kejahatan yang paling banyak belum terselesaikan yaitu `theft` dengan jumlah kasus sebanyak `21.945` kasus.

### Problem 5
5. Di bulan apa di tahun 2016 kasus pencurian (theft) sangat banyak terjadi?

In [22]:
df_theft = client.query('''
  SELECT primary_type, COUNT(primary_type) as jumlah_kejahatan, EXTRACT(MONTH FROM clearance_date) as bulan
  FROM `bigquery-public-data.austin_crime.crime`
  WHERE EXTRACT(YEAR FROM clearance_date) = 2016 AND primary_type = "Theft"
  GROUP BY primary_type, bulan
  ORDER BY jumlah_kejahatan DESC;
''').to_dataframe()
df_theft

,primary_type,jumlah_kejahatan,bulan
0,Theft,2375,8
1,Theft,2345,11
2,Theft,2185,5
3,Theft,2161,1
4,Theft,2091,12
5,Theft,2060,3
6,Theft,2038,6
7,Theft,2028,4
8,Theft,2020,10
9,Theft,1986,2


#### Insight

Kasus pencurian (theft) paling banyak terjadi pada bulan `Agustus`

### Problem 6
6. Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?

In [24]:
df_district = client.query('''
  SELECT district, COUNT(primary_type) as jumlah_kejahatan, EXTRACT(YEAR FROM clearance_date) as tahun
  FROM `bigquery-public-data.austin_crime.crime`
  WHERE EXTRACT(YEAR FROM clearance_date) = 2016 AND primary_type = "Theft"
  GROUP BY district, tahun
  ORDER BY jumlah_kejahatan DESC;
''').to_dataframe()
df_district

,district,jumlah_kejahatan,tahun
0,D,3839,2016
1,B,3422,2016
2,A,2929,2016
3,I,2898,2016
4,F,2857,2016
5,E,2836,2016
6,H,2412,2016
7,G,2111,2016
8,C,1609,2016
9,AP,119,2016


#### Insight

Tidak kejahatan pencurian terjadi paling banyak pada distrik `D` dengan jumlah  `3.839` kasus.

### Problem 7
7. Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016?

In [25]:
df_avg = client.query('''
  SELECT AVG(rata_rata.selisi_hari) as rata_rata #mencari rata - rata
  FROM(
    SELECT clearance_date, timestamp, DATE_DIFF(clearance_date, timestamp, DAY) AS selisi_hari, EXTRACT(YEAR FROM clearance_date) as tahun
    FROM `bigquery-public-data.austin_crime.crime`
    WHERE EXTRACT(YEAR FROM clearance_date) = 2016 AND primary_type = "Theft"
    GROUP BY clearance_date, timestamp, tahun
  ) as rata_rata
''').to_dataframe()
df_avg

,rata_rata
0,33.841921


#### Insight

Rata - rata membutuhkan waktu `33.8 hari` untuk dapat menyelesaian sebuah kasus kejahatan pada tahun 2016.

## Kesimpulan

Problem
1. Apa saja kasus kejahatan kota Austin di tahun 2016?
2. Berapa banyak kasus kejahatan yang terjadi pada tahun 2016?
3. jenis kejahatan apa yang paling banyak terjadi pada tahun 2016?
4. Bulan apa yang kasusnya belum banyak terselesaikan?
5. Distrik apa saja yang paling banyak terjadi kasus pencurian
6. Berapa rata - rata waktu yang diperlukan untuk menyelesaiakn sebuah kasus?

Kesimpulan berdasarkan problem<br>
Pada tahun `2016`,terdapat 6 jenis kasus yang terjadi di kota Austin. Total kasus yang ditangani sebanyak `35.747` kasus. Kasus yang paling banyak terjadi adalah `pencurian` dengan total kasus `21.945`. Kasus pencurian tercatat pada bulan `Agustus` menjadi puncak kejahatan tersebut terjadi. Paling banyak terjadi pencurian pada district `D` sebanyak `3.839` kasus. Kepolisian harus meningkatkan penjagaan pada distrik - distrik yang memiliki kasus pencurian terbanyak dan sering berpatroli untuk mengurangi durasi penyelesaian kasus pencurian yang memakan waktu kurang lebih `1 bulan`.